In [25]:
import pandas as pd
import numpy as np

from tqdm import tqdm

import random
import math
from random import sample
import os 
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

from scipy.sparse import coo_matrix, csr_matrix, lil_matrix, vstack, hstack
from scipy import sparse

random.seed(10)

In [2]:
def read_txt(path):
    data_list = []
    with open(path) as f:
        for line in tqdm(f):
            line_data = []
            for word in line.split():
                line_data.append(float(word))
            data_list.append(line_data)
    return data_list

In [3]:
# import os


# path = 'C:/Users/bbb50/Desktop/Python/hw3/Movielens/'
# files = os.listdir(path)

# for filename in files:
#     portion = os.path.splitext(filename)
#     # 如果后缀是.dat
#     if portion[1] == ".dat":  
#         # 重新组合文件名和后缀名
#         newname = portion[0] + ".txt"   
#         os.rename(os.path.join(path, filename),os.path.join(path, newname))


In [4]:
user_movie = read_txt('C:/Users/bbb50/Desktop/Python/hw3/Movielens/user_movie.txt')
user_movie = pd.DataFrame(np.array(user_movie)[:, :3].astype(int))
user_movie.columns = ['user', 'movie', 'rate']
user_movie = user_movie.groupby("user").filter(lambda x: len(x) > 3)

user_movie.movie = user_movie.movie -1
user_movie.user = user_movie.user -1

user_list = user_movie.user.value_counts().index.to_numpy()
user_list = np.sort(user_list)

movie_list = user_movie.movie.value_counts().index.to_numpy()
movie_list = np.sort(movie_list)

100000it [00:00, 602416.96it/s]


In [5]:
user_age = read_txt('C:/Users/bbb50/Desktop/Python/hw3/Movielens/user_age.txt')
user_age = pd.DataFrame(np.array(user_age).astype(int))
user_age.columns = ['user', 'age']

user_age.user = user_age.user -1
user_age.age = user_age.age -1

age_list = user_age.age.value_counts().index.to_numpy()
age_list = np.sort(age_list)

943it [00:00, 528420.66it/s]


In [6]:
user_occupation = read_txt('C:/Users/bbb50/Desktop/Python/hw3/Movielens/user_occupation.txt')
user_occupation = pd.DataFrame(np.array(user_occupation).astype(int))
user_occupation.columns = ['user', 'occupation']

user_occupation.user = user_occupation.user - 1
user_occupation.occupation = user_occupation.occupation - 1

occ_list = user_occupation.occupation.value_counts().index.to_numpy()
occ_list = np.sort(occ_list)

943it [00:00, 1138195.30it/s]


In [7]:
user_user = read_txt('C:/Users/bbb50/Desktop/Python/hw3/Movielens/user_user(knn).txt')
user_user = pd.DataFrame(np.array(user_user)[:, :2].astype(int))
user_user.columns = ['user1', 'user2']

user_user.user1 = user_user.user1 -1
user_user.user2 = user_user.user2 -1

47150it [00:00, 477692.70it/s]


In [8]:
movie_genre = read_txt('C:/Users/bbb50/Desktop/Python/hw3/Movielens/movie_genre.txt')
movie_genre = pd.DataFrame(np.array(movie_genre).astype(int))
movie_genre.columns = ['movie', 'genre']

movie_genre.movie = movie_genre.movie - 1
movie_genre.genre = movie_genre.genre - 1

genre_list = movie_genre.genre.value_counts().index.to_numpy()
genre_list = np.sort(genre_list)

2891it [00:00, 952232.83it/s]


In [9]:
movie_movie = read_txt('C:/Users/bbb50/Desktop/Python/hw3/Movielens/user_user(knn).txt')
movie_movie = pd.DataFrame(np.array(movie_movie)[:, :2].astype(int))
movie_movie.columns = ['movie1', 'movie2']

movie_movie.movie1 = movie_movie.movie1 -1
movie_movie.movie2 = movie_movie.movie2 -1

47150it [00:00, 772122.55it/s]


In [10]:
user_movie.to_feather('C:/Users/bbb50/Desktop/Python/hw3/Movielens/user_book.ftr')
user_age.to_feather('C:/Users/bbb50/Desktop/Python/hw3/Movielens/user_age.ftr')
user_occupation.to_feather('C:/Users/bbb50/Desktop/Python/hw3/Movielens/user_occupation.ftr')
user_user.to_feather('C:/Users/bbb50/Desktop/Python/hw3/Movielens/user_user.ftr')
movie_genre.to_feather('C:/Users/bbb50/Desktop/Python/hw3/Movielens/movie_genre.ftr')
movie_movie.to_feather('C:/Users/bbb50/Desktop/Python/hw3/Movielens/movie_movie.ftr')


In [11]:
n_users = user_list.shape[0]
n_movies = movie_list.shape[0]
n_occs = occ_list.shape[0]
n_genres = genre_list.shape[0]
n_ages = age_list.shape[0]

In [12]:
u_o_dict = {}
for u in tqdm(user_list):
    if u in user_occupation.user.values:
        u_o = user_occupation[user_occupation.user == u].occupation.values[0]
        u_o_dict[u] = u_o
    else:
        u_o_dict[u] = n_occs
        
occupation = []

for u in tqdm(user_movie.user):
    occupation.append(u_o_dict[u])

100%|█████████████████████████████████████████████████████████████████████| 100000/100000 [00:00<00:00, 2172426.58it/s]


In [13]:
u_a_dict = {}
for u in tqdm(user_list):
    if u in user_age.user.values:
        u_a = user_age[user_age.user == u].age.values[0]
        u_a_dict[u] = u_a
    else:
        u_a_dict[u] = n_ages
        
age = []

for u in tqdm(user_movie.user):
    age.append(u_a_dict[u])

100%|█████████████████████████████████████████████████████████████████████| 100000/100000 [00:00<00:00, 2038930.34it/s]


In [14]:
u_u_dict = {}
for u in tqdm(user_list):
    if u in user_user.user1.values:
        u_u = user_user[user_user.user1 == u].user2.values
        u_u_dict[u] = np.sort(u_u)
    else:
        u_u_dict[u] = [n_users]
user = []

for u in tqdm(user_movie.user):
    user.append(u_u_dict[u])

100%|█████████████████████████████████████████████████████████████████████| 100000/100000 [00:00<00:00, 2081158.30it/s]


In [15]:
m_m_dict = {}
for m in tqdm(movie_list):
    if m in movie_movie.movie1.values:
        m_m = movie_movie[movie_movie.movie1 == m].movie2.values
        m_m_dict[m] = np.sort(m_m)
    else:
        m_m_dict[m] = [n_movies]
        
movie = []

for m in tqdm(user_movie.movie):
    movie.append(m_m_dict[m])

100%|█████████████████████████████████████████████████████████████████████| 100000/100000 [00:00<00:00, 2127123.17it/s]


In [16]:
m_g_dict = {}
for m in tqdm(movie_list):
    if m in movie_genre.movie.values:
        m_g = movie_genre[movie_genre.movie == m].genre.values
        m_g_dict[m] = np.sort(m_g)
    else:
        m_g_dict[m] = [n_genres]
        
genre = []

for m in tqdm(user_movie.movie):
    genre.append(m_g_dict[m])

100%|█████████████████████████████████████████████████████████████████████| 100000/100000 [00:00<00:00, 2224575.70it/s]


In [17]:
user_movie['age'] = age
user_movie['occupation'] = occupation
user_movie['genre'] = genre
user_movie['movie_inter'] = movie
user_movie['user_inter'] = user

In [18]:
user_movie.to_feather('C:/Users/bbb50/Desktop/Python/hw3/Movielens/user_movie_fea.ftr')

In [19]:
user_movie

,user,movie,rate,age,occupation,genre,movie_inter,user_inter
0,195,241,3,4,2,[4],"[14, 16, 25, 31, 65, 74, 83, 149, 156, 175, 19...","[1, 13, 18, 57, 62, 64, 77, 78, 86, 97, 173, 1..."
1,185,301,3,3,3,"[5, 9, 12, 15]","[35, 38, 39, 46, 110, 111, 133, 139, 169, 172,...","[14, 53, 55, 83, 108, 129, 134, 140, 144, 158,..."
2,21,376,1,2,2,"[3, 4]","[27, 32, 60, 98, 99, 104, 116, 133, 177, 189, ...","[4, 7, 55, 63, 94, 101, 102, 124, 157, 196, 19..."
3,243,50,2,2,0,"[7, 13, 16, 17]","[22, 41, 63, 90, 113, 123, 124, 127, 174, 181,...","[42, 61, 91, 93, 108, 127, 173, 177, 183, 193,..."
4,165,345,1,4,7,"[5, 7]","[7, 63, 93, 108, 177, 196, 221, 266, 267, 275,...","[32, 38, 60, 106, 125, 126, 128, 142, 169, 204..."
...,...,...,...,...,...,...,...,...
99995,879,475,3,1,5,[4],"[10, 18, 21, 55, 86, 108, 109, 124, 207, 209, ...","[55, 86, 91, 93, 108, 129, 177, 220, 221, 248,..."
99996,715,203,5,3,4,"[4, 14]","[2, 34, 39, 46, 106, 110, 112, 132, 133, 139, ...","[5, 6, 17, 58, 59, 84, 127, 150, 193, 209, 229..."
99997,275,1089,1,2,5,[15],[1682],"[55, 91, 93, 108, 129, 177, 221, 266, 267, 279..."
99998,12,224,2,4,7,"[3, 4]","[5, 15, 17, 47, 50, 59, 89, 90, 122, 127, 227,...","[6, 58, 63, 93, 94, 101, 150, 177, 193, 200, 2..."


In [26]:
genre_mat = lil_matrix((user_movie.shape[0], (n_genres+1)), dtype=int)
for ind in tqdm(user_movie.index.to_numpy()):
    genre_mat[ind, user_movie.genre[ind]] = 1
    
genre_mat = genre_mat.tocsr()

100%|███████████████████████████████████████████████████████████████████████| 100000/100000 [00:05<00:00, 19169.38it/s]


In [27]:
mi_mat = lil_matrix((user_movie.shape[0], (n_movies+1)), dtype=int)
for ind in tqdm(user_movie.index.to_numpy()):
    mi_mat[ind, user_movie.movie_inter[ind]] = 1
    
mi_mat = mi_mat.tocsr()

100%|███████████████████████████████████████████████████████████████████████| 100000/100000 [00:06<00:00, 16471.37it/s]


In [30]:
ui_mat = lil_matrix((user_movie.shape[0], (n_users+1)), dtype=int)
for ind in tqdm(user_movie.index.to_numpy()):
    ui_mat[ind, user_movie.user_inter[ind]] = 1
    
ui_mat = ui_mat.tocsr()

100%|███████████████████████████████████████████████████████████████████████| 100000/100000 [00:06<00:00, 15087.02it/s]


In [34]:
row  = user_movie.user.index.to_numpy()
data = np.ones(user_movie.shape[0]).astype(int)

col  = user_movie.user.values
user_mat = csr_matrix((data, (row, col)), shape=(user_movie.shape[0], n_users))

col  = user_movie.movie.values
movie_mat = csr_matrix((data, (row, col)), shape=(user_movie.shape[0], n_movies))

col  = user_movie.age.values
age_mat = csr_matrix((data, (row, col)), shape=(user_movie.shape[0], (n_ages+1)))

col  = user_movie.occupation.values
occ_mat = csr_matrix((data, (row, col)), shape=(user_movie.shape[0], (n_occs+1)))



In [40]:
um_fea_mat = hstack([user_mat, movie_mat, age_mat, occ_mat, genre_mat, ui_mat, mi_mat])
um_fea_mat = um_fea_mat.tocsr()
sparse.save_npz('C:/Users/bbb50/Desktop/Python/hw3/Movielens/um_fea_mat.npz', um_fea_mat, True) 

In [43]:
root = 'C:/Users/bbb50/Desktop/Python/hw3/Movielens'

train_val_list, test_list = [], []
test_ind_list = []

skf = StratifiedKFold(n_splits=5)
um_i = user_movie.index.to_numpy()

for train_index, test_index in skf.split(um_i, user_movie.user.to_numpy()):
    train, test =um_i[train_index], um_i[test_index]
    test_ind_list.append(test_index)
    train_val_list.append(user_movie.loc[train])
    test_list.append(user_movie.loc[test])
    
train_list, val_list = [], []
train_ind_list, val_ind_list = [], []

for df in train_val_list:
    train_ind, val_ind, _ , _ = train_test_split(df.index.to_numpy(), df.user.to_numpy(), stratify = df.user.to_numpy(), test_size = 1/8)
    
    val_list.append(df.loc[val_ind])
    train_list.append(df.loc[train_ind])    

    val_ind_list.append(val_ind)
    train_ind_list.append(train_ind)    
    

In [44]:
sub_dir =  os.path.join(root, 'train')
if not(os.path.isdir(sub_dir)):
    os.makedirs(sub_dir)
for i, ind in enumerate(train_ind_list):
    
    path = os.path.join(sub_dir, 'train_ind'+str(i)+'.npy')
    np.save(path, ind)

sub_dir =  os.path.join(root, 'test')
if not(os.path.isdir(sub_dir)):
    os.makedirs(sub_dir)
for i, ind in enumerate(test_ind_list):
    
    path = os.path.join(sub_dir, 'test_ind'+str(i)+'.npy')
    np.save(path, ind)
    
sub_dir =  os.path.join(root, 'val')
if not(os.path.isdir(sub_dir)):
    os.makedirs(sub_dir)
for i, ind in enumerate(val_ind_list):
    
    path = os.path.join(sub_dir, 'val_ind'+str(i)+'.npy')
    np.save(path, ind)